# 나에게 맞는 영화 추천하는 classifier

## Step 1. 데이터 전처리

In [1]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
import numpy as np
from implicit.als import AlternatingLeastSquares

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count1'}, inplace=True)

In [4]:
ratings['count1']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count1, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## Step 2. 데이터 분석
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [6]:
# ratings에 있는 유니크한 영화 개수
print('ratings에 있는 유니크한 영화 개수 :',ratings['movie_id'].nunique())

ratings에 있는 유니크한 영화 개수 : 3628


In [7]:
# rating에 있는 유니크한 사용자 수
print ('rating에 있는 유니크한 사용자 수 :', ratings['user_id'].nunique())

rating에 있는 유니크한 사용자 수 : 6039


In [8]:
# 인기 많은 영화 30개
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

### pd.merge를 사용하여 ratings와 movies를 합쳐 주었다.

In [9]:
movie_rating = pd.merge(ratings, movies, on ='movie_id')
movie_rating.head()

,user_id,movie_id,count1,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [10]:
# 인기 많은 영화 30개
movie_count = movie_rating.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 사용할 컬럼만 남기기
- user_id,title,count 데이터만 남겼다.

In [11]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id','title','count1' ]
movie_rating = movie_rating[using_cols]
movie_rating.head(10)

,user_id,title,count1
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5
5,18,One Flew Over the Cuckoo's Nest (1975),4
6,19,One Flew Over the Cuckoo's Nest (1975),5
7,24,One Flew Over the Cuckoo's Nest (1975),5
8,28,One Flew Over the Cuckoo's Nest (1975),3
9,33,One Flew Over the Cuckoo's Nest (1975),5


In [12]:
# 유저별 몇 개의 영화를 봤는지에 대한 통계
user_count = movie_rating.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [13]:
# 유저별 별점 중앙값에 대한 통계
user_median = movie_rating.groupby('user_id')['count1'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count1, dtype: float64

## Step 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가하기

In [14]:
# 내가 좋아하는 영화
my_favorite = ['Terminator 2: Judgment Day (1991)' , 'Men in Black (1997)' ,'Matrix, The (1999)' ,'Star Wars: Episode I - The Phantom Menace (1999)' ,'Lord of the Rings, The (1978)']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['sangmin']*5, 'title': my_favorite, 'count1':[5]*5})

if not movie_rating.isin({'user_id':['sangmin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    movie_rating = movie_rating.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

movie_rating.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,count1
836473,5851,One Little Indian (1973),5
836474,5854,Slaughterhouse (1987),4
836475,5854,"Promise, The (Versprechen, Das) (1994)",3
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4
836477,5948,Identification of a Woman (Identificazione di ...,5
0,sangmin,Terminator 2: Judgment Day (1991),5
1,sangmin,Men in Black (1997),5
2,sangmin,"Matrix, The (1999)",5
3,sangmin,Star Wars: Episode I - The Phantom Menace (1999),5
4,sangmin,"Lord of the Rings, The (1978)",5


In [15]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = movie_rating['user_id'].unique()
title_unique = movie_rating['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [16]:
print(user_to_idx['sangmin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(title_to_idx['Matrix, The (1999)'])

6039
124


In [17]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = movie_rating['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movie_rating):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie_rating['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = movie_rating['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(movie_rating):
    print('title column indexing OK!!')
    movie_rating['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

movie_rating

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,count1
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,92,5
1,6039,175,5
2,6039,124,5
3,6039,60,5


## Step 4. CSR matrix를 직접 만들기

In [18]:
from scipy.sparse import csr_matrix

num_user = movie_rating['user_id'].nunique()
num_title = movie_rating['title'].nunique()
print('')
csr_data = csr_matrix((movie_rating.count1, (movie_rating.user_id, movie_rating.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## Step 5.als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 보기

In [29]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'
print('good')

good


In [30]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)
print('good')

good


In [31]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [32]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [34]:
sangmin, Matrix = user_to_idx['sangmin'], title_to_idx['Matrix, The (1999)']
sangmin_vector, Matrix_vector = als_model.user_factors[sangmin], als_model.item_factors[Matrix]

print('슝=3')

슝=3


In [35]:
sangmin_vector

array([ 0.8807932 ,  0.9825254 ,  0.23762973,  0.4445663 , -0.40492633,
       -0.8168586 ,  0.35804978,  0.16124864,  0.9349108 ,  0.00934939,
        0.29607853, -0.28920445,  0.58539784,  0.24816073, -1.5676862 ,
       -0.47314042, -0.19038627, -1.561374  ,  0.45807838,  0.7243735 ,
        0.34375277, -0.1085745 ,  0.1028441 , -0.50906605, -0.26769197,
       -0.49804905,  0.19480509,  0.49620473,  0.19300315, -0.2745423 ,
       -0.38459337,  0.17222458, -0.31094322, -0.5141287 , -0.68695116,
        0.52829695,  0.25271916,  0.36715215, -0.58563477,  0.4312057 ,
       -0.61341584,  0.9236263 ,  0.0472956 , -0.5950955 , -0.06739752,
       -0.30845004,  0.40623367,  0.3658004 ,  0.2547705 ,  0.18642347,
        0.5829065 , -0.49337313, -0.4611494 ,  0.27569687, -0.19159688,
       -0.10627398, -0.4851296 , -0.31599683,  0.8746697 , -0.31577104,
       -0.13529287,  0.5538944 ,  0.3333899 , -0.60973233,  0.6552597 ,
        0.07538743, -0.16610797, -0.7612352 ,  0.54204565,  0.36

In [38]:
Matrix_vector

array([ 0.00157465,  0.01782867,  0.00679323,  0.0144962 , -0.01375423,
       -0.00709325,  0.01031549,  0.02249157,  0.01176983,  0.0089668 ,
       -0.00905066,  0.00167372,  0.00336437,  0.02070015, -0.02988837,
       -0.01047611,  0.02134544, -0.02446876,  0.01705289,  0.0218925 ,
        0.01935981,  0.01722661,  0.02100373, -0.00311005,  0.00751065,
       -0.00950797,  0.00020448,  0.03168873,  0.01000839,  0.02474383,
       -0.00548   , -0.00362242,  0.00391969, -0.01766503, -0.02374172,
        0.03584104, -0.00205635,  0.00687367, -0.02095524,  0.0298985 ,
       -0.01727464,  0.02172506, -0.00577886, -0.00585371,  0.00043466,
        0.01967588, -0.00134222, -0.00278377,  0.01433611,  0.00258844,
        0.02674787,  0.0003391 , -0.0109922 ,  0.02184798,  0.01051101,
       -0.00609565,  0.0105807 , -0.00835107,  0.03192262, -0.00186961,
        0.01011601,  0.03547445,  0.00962104, -0.01291507,  0.02452202,
        0.00064998,  0.00106709, -0.0059382 , -0.00479021,  0.01

0.6023625

## Step 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보기.


In [47]:
# 내가 선호하는 영화의 훈련된 모델이 예측한 나의 선호도
np.dot(sangmin_vector, Matrix_vector)

0.6023625

In [42]:
# 아무거나 고른 영화의 훈련된 모델이 예측한 나의 선호도
Forrest_Gump = title_to_idx['Forrest Gump (1994)']
Forrest_Gump_vector = als_model.item_factors[Forrest_Gump]
np.dot(sangmin_vector, Forrest_Gump_vector)

0.111863285

## Step 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [44]:
# 내가 선호하는 영화와 유사한 영화
favorite_movie ='Men in Black (1997)'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(175, 0.9999999),
 (107, 0.83594644),
 (92, 0.60567266),
 (62, 0.57161164),
 (150, 0.54065007),
 (145, 0.49253395),
 (124, 0.48248556),
 (82, 0.47747827),
 (3466, 0.4280867),
 (670, 0.42443946),
 (375, 0.41912672),
 (138, 0.412976),
 (542, 0.38622496),
 (60, 0.38190734),
 (320, 0.37762132)]

In [46]:
#title_to_idx 를 뒤집어, index로부터 영화 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Independence Day (ID4) (1996)',
 'Fifth Element, The (1997)',
 'Matrix, The (1999)',
 'Lost World: Jurassic Park, The (1997)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Galaxy Quest (1999)',
 'Face/Off (1997)',
 'True Lies (1994)',
 'Rocky Horror Picture Show, The (1975)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Bob Roberts (1992)']

In [48]:
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [49]:
get_similar_movie('Independence Day (ID4) (1996)')

['Independence Day (ID4) (1996)',
 'Deep Impact (1998)',
 'Stargate (1994)',
 'Mars Attacks! (1996)',
 'Armageddon (1998)',
 'Men in Black (1997)',
 'Starship Troopers (1997)',
 'Fifth Element, The (1997)',
 'Twister (1996)',
 'Face/Off (1997)']

## Step 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [50]:
user = user_to_idx['sangmin']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(107, 0.6698302),
 (62, 0.46412975),
 (64, 0.41211528),
 (44, 0.39222434),
 (87, 0.34638232),
 (117, 0.29381046),
 (670, 0.28498262),
 (141, 0.2832936),
 (82, 0.28222743),
 (145, 0.27255526),
 (38, 0.26904136),
 (200, 0.26423922),
 (150, 0.26128048),
 (375, 0.25568074),
 (75, 0.20630628),
 (99, 0.19532597),
 (48, 0.19374797),
 (372, 0.19103578),
 (58, 0.19103009),
 (179, 0.1837028)]

In [51]:
[idx_to_title[i[0]] for i in movie_recommended]

['Jurassic Park (1993)',
 'Total Recall (1990)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Braveheart (1995)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Galaxy Quest (1999)',
 'Fugitive, The (1993)',
 'Lost World: Jurassic Park, The (1997)',
 'Fifth Element, The (1997)',
 'Sixth Sense, The (1999)',
 'Terminator, The (1984)',
 'Independence Day (ID4) (1996)',
 'Face/Off (1997)',
 'Hunt for Red October, The (1990)',
 'American Beauty (1999)',
 'Saving Private Ryan (1998)',
 'X-Men (2000)',
 'Mission: Impossible (1996)',
 'Rock, The (1996)']

## 기여도 확인

In [52]:
Total_Recall = title_to_idx['Total Recall (1990)']
explain = als_model.explain(user, csr_data, itemid=Total_Recall)

In [53]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Terminator 2: Judgment Day (1991)', 0.18113735436938658),
 ('Matrix, The (1999)', 0.14412815333867385),
 ('Men in Black (1997)', 0.12141769069022541),
 ('Star Wars: Episode I - The Phantom Menace (1999)', 0.018281253612579915),
 ('Lord of the Rings, The (1978)', -0.0113966720155779)]

# 마치며...   
-  영화 제목을 읽기 위한 데이터를 불러올 때 pd.read_csv() 함수에서 'utf-8' codec can't decode byte 0xe9 in position 3114: invalid continuation byte' 오류가 나서 encoding = 'ISO-8859-1'을 추가하여 오류를 해결했다.
- 처음에 노드에서 공부한 음악 추천 데이터는 한 데이터 셋에 모두 정보가 있었지만 영화 추천 데이터는 영화 별점에 대한 데이터와 영화 제목에 대한 데이터가 따로 나누어져 있어서 어떻게 할 지 고민하다가 pd.merge() 함수를 써서 합치니 모델이 문제 없이 잘 영화추천을 해주었다. 그리고 영화 별점에 대한 데이터인 'rating' 항목을 'count'로 바꿔서 데이터셋 설정을 했었는데 pandas에서 제공하는 '.count' 기능과 겹쳐서 'movie_rating.count' 이 부분에서 'TypeError: len() of unsized object'오류가 나서 'movie_rating.count'이 부분 값을 찍어보니 columns 가 1이여야 하는데 3이 찍혀있는 것을 알았다. 그래서 'count1'로 항목 이름을 바꿨더니 오류가 해결되었다. 